In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy
import tensorflow as tf
from tensorflow.python.framework import ops
from hpf_utils import *

C:\Users\shubhamkumar.singh\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(391, 6)
here2:  (2, 391)
shape of input data:  (391, 227, 227, 3)
shape of target variable:  (391, 2)
shape of y_coord:  (391, 42)
shape of y_visibility:  (391, 21)
shape of y_pose:  (391, 3)
shape of y_gender:  (391, 2)


In [2]:
X_train, Y_train, y_coord, y_visibility, y_pose, y_gender = load_data()

X_test = X_train[int(0.8*len(X_train)):]

Y_test = Y_train[int(0.8*len(Y_train)):]

X_train = X_train[0:int(0.8*len(X_train))]
print(Y_train.shape)
Y_train = Y_train[0:int(0.8*len(Y_train))]

gender_train = y_gender[0:int(0.8*len(y_gender))]
gender_test  = y_gender[int(0.8*len(y_gender)):0]

(391, 6)
here2:  (2, 391)
shape of input data:  (391, 227, 227, 3)
shape of target variable:  (391, 2)
shape of y_coord:  (391, 42)
shape of y_visibility:  (391, 21)
shape of y_pose:  (391, 3)
shape of y_gender:  (391, 2)
(391, 2)


In [3]:
print(Y_train.shape[0])

312


In [4]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, [None, 227, 227, 3])
    Y = tf.placeholder(tf.float32,[None, 2])
    y_coord = tf.placeholder(tf.float32, [None, 42])
    y_visibility = tf.placeholder(tf.float32, [None, 21])
    y_pose = tf.placeholder(tf.float32, [None, 3])
    y_gender = tf.placeholder(tf.float32, [None, 2])
    
    return X, Y, y_coord, y_visibility, y_pose, y_gender

In [5]:
def initialize_parameters():
    tf.set_random_seed(1)
    W1  = tf.get_variable("W1", [11,11, 3,96], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2  = tf.get_variable("W2", [5,5, 96,256], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W3  = tf.get_variable("W3", [3,3,256,384], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W4  = tf.get_variable("W4", [3,3,384,384], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W5  = tf.get_variable("W5", [3,3,384,256], initializer=tf.contrib.layers.xavier_initializer(seed=0))

    W1A   = tf.get_variable("W1A",[4,4, 96,256], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W3A   = tf.get_variable("W3A",[2,2,384,256], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W_ALL = tf.get_variable("W_ALL",[1,1,768,192], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    #W1A = tf.get_variable("W1A",[4,4, 96,256], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters = {"W1":W1,
                 "W2":W2,
                 "W3":W3,
                 "W4":W4,
                 "W5":W5,
                 "bc1": tf.Variable(tf.constant(0.0, shape=[96]),        name="bc1"),
                 "bc2": tf.Variable(tf.constant(1.0, shape=[256]),       name="bc2"),
                 "bc3": tf.Variable(tf.constant(0.0, shape=[384]),       name="bc3"),
                 "bc4": tf.Variable(tf.constant(1.0, shape=[384]),       name="bc4"),
                 "bc5": tf.Variable(tf.constant(1.0, shape=[256]),       name="bc5"),
                 #"bf1": tf.Variable(tf.constant(1.0, shape=[120]),       name="bf1"),
                 #"bf2": tf.Variable(tf.constant(1.0, shape=[84]),        name="bf2"),
                 #"bf3": tf.Variable(tf.constant(1.0, shape=[2]),         name="bf3")
                 "W1A": W1A,
                 "W3A": W3A,
                 "W_ALL": W_ALL}

    return parameters
    

In [6]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    W5 = parameters['W5']
    bc1 = parameters['bc1']
    bc2 = parameters['bc2']
    bc3 = parameters['bc3']
    bc4 = parameters['bc4']
    bc5 = parameters['bc5']
    W1A = parameters['W1A']
    W3A = parameters['W3A']
    W_ALL = parameters['W_ALL']
    
    #layer1
    X1 = tf.nn.conv2d(X, W1, strides = [1,4,4,1], padding ='VALID')
    Z1 = tf.nn.bias_add(X1, bc1)
    A1 = tf.nn.relu(Z1)
    print("A1: ", A1)
    L1 = tf.nn.local_response_normalization(A1, depth_radius=5.0, bias=2.0, alpha=1e-4, beta=0.75)
    P1 = tf.nn.max_pool(L1, ksize=[1,3,3,1],strides=[1,2,2,1], padding='VALID')
    print("P1: ", P1)
    #layer2
    X2 = tf.nn.conv2d(P1, W2, strides = [1,1,1,1], padding ='SAME')
    Z2 = tf.nn.bias_add(X2, bc2)
    A2 = tf.nn.relu(Z2)
    print("A2: ", A2)
    L2 = tf.nn.local_response_normalization(A2, depth_radius=5.0, bias=2.0, alpha=1e-4, beta=0.75)
    P2 = tf.nn.max_pool(L2, ksize=[1,3,3,1],strides=[1,2,2,1], padding='VALID')
    print("P2: ", P2)
    #layer3
    X3 = tf.nn.conv2d(P2, W3, strides = [1,1,1,1], padding ='SAME')
    print("X3: ", X3)
    Z3 = tf.nn.bias_add(X3, bc3)
    P3 = tf.nn.relu(Z3)
    #layer4
    X4 = tf.nn.conv2d(P3, W4, strides = [1,1,1,1], padding ='SAME')
    print("X4: ", X4)
    Z4 = tf.nn.bias_add(X4, bc4)
    P4 = tf.nn.relu(Z4)
    #layer5
    X5 = tf.nn.conv2d(P4, W5, strides = [1,1,1,1], padding ='SAME')
    print("X5: ", X5)
    Z5 = tf.nn.bias_add(X5, bc5)
    A5 = tf.nn.relu(Z5)
    P5 = tf.nn.max_pool(A5, ksize=[1,3,3,1],strides=[1,2,2,1], padding='VALID')
    print("P5: ", P5)
    #layer1a
    Z1A = tf.nn.conv2d(P1, W1A, strides = [1,4,4,1], padding ='VALID')
    print("Z1A: ", Z1A)
    A1A = tf.nn.relu(Z1A)
    #layer3a
    Z3A = tf.nn.conv2d(P3, W3A, strides = [1,2,2,1], padding ='VALID')
    print("Z3A: ", Z3A)
    A3A = tf.nn.relu(Z3A)
    
    #concat
    concat = tf.concat([A1A, A3A, P5], 3)
    print("concat: ", concat)
    #CONV_ALL
    Z_ALL = tf.nn.conv2d(concat, W_ALL, strides = [1,1,1,1], padding ='VALID')
    A_ALL = tf.nn.relu(Z_ALL)
    
    P = tf.contrib.layers.flatten(A_ALL)
    #fully_connected_1
    shape_fc =int(np.prod(A_ALL.get_shape()[1:]))
    print("shape: ", shape_fc)
    fc_full_w = tf.Variable(tf.truncated_normal(shape = (6912,3072), mean = 0, stddev = 0.1))
    fc_full_b = tf.Variable(tf.zeros(3072))
    fc_full_z = tf.matmul(P,fc_full_w) + fc_full_b
    fc_full_a = tf.nn.relu(fc_full_z)
    
    #face/non_face
    fc_det_w  = tf.Variable(tf.truncated_normal(shape = (3072,512), mean = 0, stddev = 0.1))
    fc_det_b  = tf.Variable(tf.zeros(512))
    fc_det_z  = tf.matmul(fc_full_a,fc_det_w) + fc_det_b
    fc_det_a  = tf.nn.relu(fc_det_z)
    
    #genger_m/f
    fc_gen_w  = tf.Variable(tf.truncated_normal(shape = (3072,512), mean = 0, stddev = 0.1))
    fc_gen_b  = tf.Variable(tf.zeros(512))
    fc_gen_z  = tf.matmul(fc_full_a,fc_gen_w) + fc_gen_b
    fc_gen_a  = tf.nn.relu(fc_gen_z)
    
    #output_face_detection
    face_w = tf.Variable(tf.truncated_normal(shape = (512, 2), mean = 0 , stddev = 0.1))
    face_b = tf.Variable(tf.zeros(2))
    face_z = tf.matmul(fc_det_a, face_w) + face_b
    
    #output_gender
    gen_w = tf.Variable(tf.truncated_normal(shape = (512, 2), mean = 0 , stddev = 0.01))
    gen_b = tf.Variable(tf.zeros(2))
    gen_z = tf.matmul(fc_gen_a, gen_w) + gen_b
    
    a = tf.Print(gen_z, [gen_z], message="This is a: ")
    aa = tf.Print(face_z,[face_z], message = "This is aa: ")
    return face_z, gen_z

In [7]:
def compute_cost(Z, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z, labels=Y))
    
    return cost

In [8]:
def model(X_train,Y_train,X_test,Y_test, gender_train, gender_test, learning_rate=0.0003,
         num_epochs=16, minibatch_size=32,print_cost=True):
    
    ops.reset_default_graph()
    tf.set_random_seed(1)
    seed=3
    
    (m,n_H0,n_W0,n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    n_g = gender_train.shape[1]
    
    print("n_y: ", n_y)
    print("n_g: ", n_g)
    
    costs   = []
    g_costs = []
    
    X, Y, y_coord, y_visibility, y_pose, y_gender = create_placeholders(n_H0, n_W0, n_C0, n_y)
    parameters = initialize_parameters()
    
    face, gender = forward_propagation(X, parameters)
    
    cost   = compute_cost(face, Y)
    g_cost = compute_cost(gender, y_gender)
    
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    optimizer = tf.train.MomentumOptimizer(1e-4, 0.9, use_nesterov=True).minimize(cost+g_cost)
    #g_optimizer = tf.train.MomentumOptimizer(1e-4, 0.9, use_nesterov=True).minimize(g_cost)
    #g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(num_epochs):
            minibatch_cost = 0.
            g_minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            print("gender_train: ", gender_train.shape)
            print("y shape: ", Y_train.shape)
            minibatches = random_mini_batches(X_train, Y_train, gender_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                #print("popo: ", face.eval())
                (minibatch_X, minibatch_Y, minibatch_g) = minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y, y_gender:minibatch_g})
               # _ , temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
               # _ , gen_cost  = sess.run([g_optimizer, g_cost], feed_dict={X:minibatch_X, y_gender:minibatch_g})
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                #g_minibatch_cost += gen_cost / num_minibatches
                

            #print("after epoch")
            # Print the cost every epoch
            if print_cost == True and epoch % 1 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
                #print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
                g_costs.append(g_minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        
        #plt.plot(np.squeeze(g_costs))
        #plt.ylabel('cost')
        #plt.xlabel('iterations (per tens)')
        #plt.title("Learning rate =" + str(learning_rate))
        #plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(face, 1)
        predict_gen = tf.argmax(gender, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        g_correct_prediction = tf.equal(predict_gen, tf.argmax(y_gender, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        g_accuracy = tf.reduce_mean(tf.cast(g_correct_prediction, "float"))
        print(g_accuracy)
        
        face_train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        face_test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Face Train Accuracy:", face_train_accuracy)
        print("Face Test Accuracy:", face_test_accuracy)
        
        gender_train_accuracy = g_accuracy.eval({X: X_train, Y: gender_train})
        gender_test_accuracy = accuracy.eval({X: X_test, Y: gender_test})
        print("Face Train Accuracy:", gender_train_accuracy)
        print("Face Test Accuracy:", gender_test_accuracy)
            
            
        return train_accuracy, test_accuracy, parameters

In [9]:
_, _, parameters = model(X_train,Y_train,X_test,Y_test, gender_train, gender_test)

n_y:  2
n_g:  2
A1:  Tensor("Relu:0", shape=(?, 55, 55, 96), dtype=float32)
P1:  Tensor("MaxPool:0", shape=(?, 27, 27, 96), dtype=float32)
A2:  Tensor("Relu_1:0", shape=(?, 27, 27, 256), dtype=float32)
P2:  Tensor("MaxPool_1:0", shape=(?, 13, 13, 256), dtype=float32)
X3:  Tensor("Conv2D_2:0", shape=(?, 13, 13, 384), dtype=float32)
X4:  Tensor("Conv2D_3:0", shape=(?, 13, 13, 384), dtype=float32)
X5:  Tensor("Conv2D_4:0", shape=(?, 13, 13, 256), dtype=float32)
P5:  Tensor("MaxPool_2:0", shape=(?, 6, 6, 256), dtype=float32)
Z1A:  Tensor("Conv2D_5:0", shape=(?, 6, 6, 256), dtype=float32)
Z3A:  Tensor("Conv2D_6:0", shape=(?, 6, 6, 256), dtype=float32)
concat:  Tensor("concat:0", shape=(?, 6, 6, 768), dtype=float32)
shape:  6912
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

gender_train:  (312, 2)
y shape:  (312, 2)
m:  312
Cost after epoch 0: 57.4

KeyboardInterrupt: 

In [ ]:
print(X_train[0].shape)